In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image

Importieren relevanter Bibliotheken

In [ ]:
data = pd.read_csv('../data/train.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

Datensatz lesen. Konvertieren des Datensatzes in ein zweidimensionales Numpy-Array. Dann mischen wir die Daten, um die Zufälligkeit für das Training zu gewährleisten

In [ ]:
data_dev = data[0:1000].T # Daten aufteilen
Y_dev = data_dev[0] # Labels extrahieren
X_dev = data_dev[1:n] # Features extrahieren
X_dev = X_dev / 255. # Normalisierung

data_train = data[1000:m].T # Daten aufteilen
Y_train = data_train[0] # Labels extrahieren
X_train = data_train[1:n] # Features extrahieren
X_train = X_train / 255. # Normalisierung
_,m_train = X_train.shape # Größe des Trainingsdatensatzes bestimmen

Hier teilen wir das Dataset in ein Entwicklungssatz und ein Trainingssatz auf. Die Daten werden auch transponiert, um der erwarteten Eingabeform des neuronalen Netzwerks zu entsprechen. (Features als Spalten und Beispiele als Zeilen) Schließlich werden die Pixelwerte durch Teilen durch 255 normalisiert, sodass sie zwischen 0 und 1 liegen. Diese Normalisierung ist für die Stabilität des Trainings und eine schnellere Konvergenz unerlässlich.

In [ ]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5 # Für die erste versteckte Schicht
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5  # Für die zweite versteckte Schicht
    b2 = np.random.rand(10, 1) - 0.5
    W3 = np.random.rand(10, 10) - 0.5  # Für die Ausgabeschicht
    b3 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

Unser Netzwerk besteht also aus vier Schichten. Die Eingabeschicht, zwei versteckte Schichten, und die Ausgabeschicht. Wir initialisieren die Werte mithilfe von np.random.rand() als Werte zwischen 0 und 1. Indem wir den Bereich auf [-0.5, 0.5] verschieben, erreichen wir eine schneller Konvergenz. (Da zu große Werte dazu führen können, dass der Gradient "explodiert" und somit das Training instabil wird, zu kleine Werte dazu, dass der Gradient "verschwindend" klein wird und somit das Training sehr langsam wird)


W1: 10 Neuronen in der ersten versteckten Schicht, jedes mit 784 Eingangsmerkmalen/Gewichten (eben die 28x28 Pixel)
b1: Bias-Werte für die 10 Neuronen der ersten Schicht

W2, b2 und W3, b3 ähnlich, ebenfalls 10 Neuronen pro Schicht


Die Initiliasierung der Gewichte und Verzerrungen kann einen großen Einfluss darauf haben, wie gut und wie schnell ein neuronales Netzwerk während des Trainings konvergiert. Es gibt viele fortschrittliche Methoden zur Initialisierung, aber der hier gezeigte Ansatz mit kleinen zufälligen Werten ist ein einfacher und oft verwendeter Ansatz, insbesondere für kleinere Netzwerke oder zum Einstieg.

In [ ]:
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

Die beiden Aktivierungsfunktionen, die unser Neuronales Netzwerk benutzt:

    - ReLU: Benutzen wir für die zwei versteckten Schichten. Alle Werte, die kleiner als 0 sind, werden zu 0, und alle Werte, die größer als 0 sind, bleiben unverändert.

    - Softmax: Benutzen wir in der Ausgabeschicht. Sie fungiert als Wahrscheinlichkeitsverteilung, die Werte liegen also zwischen 0 und 1 und die Summe aller Werte ist genau 1. Mit ihrer Hilfe entscheidet sich das Netzwerk in der Ausgabeschicht für eines der 10 Neuronen (Maximum).

In [ ]:
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)  # ReLU für die zweite versteckte Schicht
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

Diese Funktion führt die Vorwärtspropagation des Netzwerks durch.

Sie berechnet anhand der aktuellen Gewichte und Verzerrungen aller Schichten des Netzwerks die entsprechenden Ausgaben bzw. Aktivierungen für X. Z1 entspricht also dem Produkt aus den Gewichten W1 und der Eingabe X mit anschließender Hinzufügung der Verzerrung b1. Außerdem bestimmen wir dann die Aktivierung A1 der ersten Schicht für X anhand der gewichteten Summe Z1.

Das gleiche passiert für die zweite Schicht.

Die dritte Schicht unterscheidet sich bloß in der Aktivierungsfunktion, wie bereits erklärt.

Als Rückgabe erhalten wir also die gewichteten Summen und Aktivierungen aller Schichten für die Eingabe X.

In [ ]:
# wenn Input (Z) kleiner gleich 0 => false, sonst true
def ReLU_deriv(Z):
    return Z > 0

# 1-aus-n-Code, stellt Dezimalzahlen als Binärzahlen da
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y



def backward_prop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y, m):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, db1, dW2, db2, dW3, db3


Das Ziel der Rückwertpropagation ist es, zu verstehen, wie sich eine Änderung der Gewichte und Verzerrungen auf den Gesamtfehler des Netzwerks auswirkt. Die zurückgegebenen Werte (dW1, db1, dW2, db2) stellen Gradienten dar, die die Richtung und Größe der zur Fehlerreduktion erforderlichen Änderungen anzeigen. Anders gesagt, gibt der Gradient die Richtung an, in welcher die Funktion, also die Fehlerquote, am steilsten ansteigt.

In [ ]:
def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    W3 -= alpha * dW3
    b3 -= alpha * db3
    return W1, b1, W2, b2, W3, b3

Diese Funktion aktualisiert die Gewichte und Verzerrungen des Netzwerks in Richtung des negativen Gradienten. Dieser iterative Prozess hilft dem Netzwerk, aus seinen Fehlern zu lernen. 'alpha' ist die Lernrate, die die Schrittgröße jeder Aktualisierung bestimmt.

In [ ]:
def get_predictions(A3):
    return np.argmax(A3, 0)


def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size


def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W2, W3, X, Y, m_train)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3

Diese Funktion führt das Training unseres neuronalen Netzwerks durch. Es verwendet den Gradientenabstieg, um die Gewichte und Verzerrungen kontinuierlich zu aktualisieren. Am Ende des Trainings sollte unser Netzwerk besser auf die Daten abgestimmt sein und genauere Vorhersagen treffen können.

Die Analogie mit einer Kugel, die einen Hang hinunterrollt:

    - Die Kugel repräsentiert unsere aktuelle Position (oder den aktuellen Wert der Gewichte) im Fehlerlandschaft.
    
    - Der Hang repräsentiert die Verlustfunktion.
    
    - Die Schwerkraft zwingt die Kugel dazu, den Weg des geringsten Widerstands zu suchen und sich bergab zu bewegen.


    - Eine hohe Lernrate (α) würde bedeuten, dass die Kugel einen großen Sprung bergab macht. Dies könnte zwar schneller zum Tal (dem Minimum) führen, birgt aber auch das Risiko, dass die Kugel das Tal überquert und auf der anderen Seite wieder nach oben rollt. In anderen Worten, eine zu hohe Lernrate kann dazu führen, dass der Algorithmus "überschwingt" und nicht konvergiert.
    
    - Eine niedrige Lernrate würde bedeuten, dass die Kugel kleinere Schritte bergab macht. Dies kann dazu führen, dass die Kugel sicherer und stetiger zum Tal gelangt, aber es könnte sehr lange dauern. Eine zu niedrige Lernrate kann den Trainingsprozess erheblich verlangsamen.

In [24]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 0.10, 300)

Iteration:  0
[7 6 5 ... 5 5 3] [5 8 5 ... 7 8 4]
0.10492682926829268
Iteration:  10
[7 2 2 ... 7 3 2] [5 8 5 ... 7 8 4]
0.11146341463414634
Iteration:  20
[7 2 2 ... 7 3 2] [5 8 5 ... 7 8 4]
0.18885365853658537
Iteration:  30
[7 4 2 ... 5 6 6] [5 8 5 ... 7 8 4]
0.2374390243902439
Iteration:  40
[1 4 0 ... 5 6 4] [5 8 5 ... 7 8 4]
0.3024390243902439
Iteration:  50
[1 4 0 ... 5 6 4] [5 8 5 ... 7 8 4]
0.3987317073170732
Iteration:  60
[1 4 0 ... 5 7 4] [5 8 5 ... 7 8 4]
0.46070731707317075
Iteration:  70
[1 4 0 ... 5 5 4] [5 8 5 ... 7 8 4]
0.5155365853658537
Iteration:  80
[1 4 0 ... 5 8 4] [5 8 5 ... 7 8 4]
0.5627560975609756
Iteration:  90
[1 4 0 ... 5 8 4] [5 8 5 ... 7 8 4]
0.597609756097561
Iteration:  100
[1 7 0 ... 5 8 4] [5 8 5 ... 7 8 4]
0.6253658536585366
Iteration:  110
[1 7 0 ... 5 8 4] [5 8 5 ... 7 8 4]
0.646609756097561
Iteration:  120
[1 7 0 ... 5 8 4] [5 8 5 ... 7 8 4]
0.6659024390243903
Iteration:  130
[1 7 0 ... 5 8 4] [5 8 5 ... 7 8 4]
0.6827317073170732
Iteration:  140

In [ ]:
def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    return predictions

def test_prediction(index, W1, b1, W2, b2, W3, b3):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2, W3, b3)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(0, W1, b1, W2, b2, W3, b3)
test_prediction(1, W1, b1, W2, b2, W3, b3)
test_prediction(2, W1, b1, W2, b2, W3, b3)
test_prediction(3, W1, b1, W2, b2, W3, b3)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2, W3, b3)
get_accuracy(dev_predictions, Y_dev)

In [ ]:
def display_wrong_predictions(X, Y, dev_predictions, W1, b1, W2, b2, W3, b3, num_samples=10):
    wrong_indices = np.where(dev_predictions != Y)[0]  # Find where predictions don't match the actual labels
    displayed = 0
    
    for index in wrong_indices:
        if displayed >= num_samples:  # Limit the number of displayed images to num_samples
            break

        current_image = X[:, index, None]
        predicted_label = dev_predictions[index]
        actual_label = Y[index]

        print("Predicted Label:", predicted_label)
        print("Actual Label:", actual_label)

        current_image = current_image.reshape((28, 28)) * 255
        plt.gray()
        plt.imshow(current_image, interpolation='nearest')
        plt.show()

        displayed += 1

In [ ]:
def predict_image(image_path, W1, b1, W2, b2, W3, b3):
    # Load the image
    image = Image.open(image_path).convert("L")  # Convert image to grayscale
    image = image.resize((28, 28))  # Resize image to match input size
    
    # Convert image to numpy array and normalize
    image_arr = np.array(image) / 255.0
    
    # Reshape the image to be a flat vector and then a column vector
    image_arr = image_arr.reshape((-1, 1))
    
    # Predict
    prediction = make_predictions(image_arr, W1, b1, W2, b2, W3, b3)
    
    # Display the image
    plt.gray()
    plt.imshow(np.array(image), interpolation='nearest')
    plt.title(f"Predicted Label: {prediction[0]}")
    plt.show()

    return prediction[0]

In [ ]:
display_wrong_predictions(X_dev, Y_dev, dev_predictions, W1, b1, W2, b2, W3, b3)

In [ ]:
image_path = "../data/7_black.png"
predict_image(image_path, W1, b1, W2, b2, W3, b3)